In [ ]:
from collections import defaultdict

class FPNode:
    def __init__(self, item, count=1, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.next_node = None

def build_fp_tree(transactions, min_support):
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    
    item_counts = {item: count for item, count in item_counts.items() if count >= min_support}
    if not item_counts:
        return None, None
    
    root = FPNode(None)
    header_table = {item: None for item in item_counts}
    
    for transaction in transactions:
        filtered_items = [item for item in transaction if item in item_counts]
        filtered_items.sort(key=lambda x: item_counts[x], reverse=True)
        update_fp_tree(filtered_items, root, header_table)
    
    return root, header_table

def update_fp_tree(items, node, header_table):
    if not items:
        return
    
    first_item = items[0]
    if first_item in node.children:
        child_node = node.children[first_item]
        child_node.count += 1
    else:
        child_node = FPNode(first_item, parent=node)
        node.children[first_item] = child_node
        if header_table[first_item] is None:
            header_table[first_item] = child_node
        else:
            current = header_table[first_item]
            while current.next_node:
                current = current.next_node
            current.next_node = child_node
    
    update_fp_tree(items[1:], child_node, header_table)

def mine_fp_tree(header_table, min_support, prefix, frequent_itemsets):
    for item, node in sorted(header_table.items(), key=lambda x: x[1].count if x[1] else 0):
        if node is None:
            continue
        
        new_prefix = prefix + [item]
        
        # Aggregate the support count of the current item across its linked nodes
        total_support = 0
        current_node = node
        while current_node is not None:
            total_support += current_node.count
            current_node = current_node.next_node
        
        if total_support >= min_support:
            frequent_itemsets.append((new_prefix, total_support))
            
            conditional_pattern_base = []
            current_node = node
            while current_node:
                prefix_path = []
                parent = current_node.parent
                while parent and parent.item:
                    prefix_path.append(parent.item)
                    parent = parent.parent
                if prefix_path:
                    conditional_pattern_base.extend([prefix_path] * current_node.count)
                current_node = current_node.next_node
            
            conditional_tree, new_header_table = build_fp_tree(conditional_pattern_base, min_support)
            if new_header_table:
                mine_fp_tree(new_header_table, min_support, new_prefix, frequent_itemsets)

def fp_growth(transactions, min_support):
    root, header_table = build_fp_tree(transactions, min_support)
    if not root:
        return []
    frequent_itemsets = []
    mine_fp_tree(header_table, min_support, [], frequent_itemsets)
    return frequent_itemsets

# Example usage
transactions = [
    ['milk', 'bread', 'butter'],
    ['bread', 'butter'],
    ['milk', 'bread'],
    ['milk', 'butter'],
    ['bread', 'butter']
]

min_support = 2
frequent_itemsets = fp_growth(transactions, min_support)

# Print the frequent itemsets
print("Frequent Itemsets:")
for itemset, count in frequent_itemsets:
    print(f"Itemset: {itemset}, Support: {count}")
